In [3]:
!pip install nltk
!pip install category_encoders
!pip install pymorphy3


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/54.1 kB ? eta -:--:--
   ------- -------------------------------- 10.2/54.1 kB ? eta -:--:--
   ---------------------- ----------------- 30.7/54.1 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 54.1/54.1 kB 402.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.4 MB ? eta -:--:--
    --------------------------------------- 0.2/8.4 MB 4.1 MB/s eta 0:00:02
    --------------------------------------- 0.2/8.4 MB 4.1 MB/s eta 0:00:02
   - -------------------------------------- 0.4/8.4 MB 2.2 MB/s eta 0:00:04
   -- ------------------------------------- 0.4/8.4 MB 2.0 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/8.4 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.4 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.1/8.4 MB 3.2 MB/s eta 0:00:03
   ----- --------------------

ERROR: Could not install packages due to an OSError: [WinError 2] Не удается найти указанный файл: 'C:\\Python312\\Scripts\\pymorphy.exe' -> 'C:\\Python312\\Scripts\\pymorphy.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import unicodedata
from pymorphy3 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import OneHotEncoder
from category_encoders import HashingEncoder, TargetEncoder
from sklearn.model_selection import KFold
import numpy as np

In [5]:
df = pd.read_csv('../data/ml_ozon_сounterfeit_train.csv')
df.describe()

,id,resolution,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,photos_published_count,videos_published_count,...,ExemplarReturnedCountTotal30,ExemplarReturnedCountTotal90,ExemplarReturnedValueTotal7,ExemplarReturnedValueTotal30,ExemplarReturnedValueTotal90,ItemVarietyCount,ItemAvailableCount,seller_time_alive,ItemID,SellerID
count,197198.000000,197198.000000,47193.000000,47193.000000,47193.000000,47193.000000,47193.000000,47193.000000,47193.000000,47193.000000,...,189268.000000,189791.000000,187007.000000,189268.000000,189791.000000,196201.000000,196201.000000,197198.000000,197198.000000,197198.000000
mean,225331.179292,0.066187,2.048355,0.594876,1.109932,1.253067,13.446274,9.519505,4.412222,0.482699,...,51.802196,147.169818,701.520465,905.735423,1025.111018,98.790485,98.732489,684.575827,110563.014001,1504.121553
std,130672.598024,0.248610,13.767362,3.398610,6.044903,6.175065,63.103940,46.637693,18.477008,3.595718,...,164.435978,511.811000,413.671912,374.229445,360.664928,245.133031,245.151594,498.399100,63773.111390,2256.212708
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,112168.250000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,3.000000,9.000000,611.486957,858.718884,968.864810,2.000000,2.000000,262.000000,55220.250000,157.000000
50%,225326.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,...,16.000000,45.000000,856.211201,1008.683523,1116.673313,7.000000,7.000000,607.000000,110732.500000,514.000000
75%,336923.250000,0.000000,1.000000,0.000000,1.000000,1.000000,7.000000,4.000000,3.000000,0.000000,...,52.000000,143.000000,990.670761,1131.163329,1229.895511,58.000000,58.000000,1058.000000,165852.750000,1723.000000
max,455045.000000,1.000000,1007.000000,199.000000,329.000000,518.000000,4465.000000,1753.000000,1035.000000,241.000000,...,55681.000000,181600.000000,1663.665513,1807.589625,1914.000284,1548.000000,1548.000000,2265.000000,220806.000000,12508.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197198 entries, 0 to 197197
Data columns (total 45 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id                            197198 non-null  int64  
 1   resolution                    197198 non-null  int64  
 2   brand_name                    116667 non-null  object 
 3   description                   171138 non-null  object 
 4   name_rus                      197198 non-null  object 
 5   CommercialTypeName4           197198 non-null  object 
 6   rating_1_count                47193 non-null   float64
 7   rating_2_count                47193 non-null   float64
 8   rating_3_count                47193 non-null   float64
 9   rating_4_count                47193 non-null   float64
 10  rating_5_count                47193 non-null   float64
 11  comments_published_count      47193 non-null   float64
 12  photos_published_count        47193 non-null

In [7]:
df.columns

Index(['id', 'resolution', 'brand_name', 'description', 'name_rus',
       'CommercialTypeName4', 'rating_1_count', 'rating_2_count',
       'rating_3_count', 'rating_4_count', 'rating_5_count',
       'comments_published_count', 'photos_published_count',
       'videos_published_count', 'PriceDiscounted', 'item_time_alive',
       'item_count_fake_returns7', 'item_count_fake_returns30',
       'item_count_fake_returns90', 'item_count_sales7', 'item_count_sales30',
       'item_count_sales90', 'item_count_returns7', 'item_count_returns30',
       'item_count_returns90', 'GmvTotal7', 'GmvTotal30', 'GmvTotal90',
       'ExemplarAcceptedCountTotal7', 'ExemplarAcceptedCountTotal30',
       'ExemplarAcceptedCountTotal90', 'OrderAcceptedCountTotal7',
       'OrderAcceptedCountTotal30', 'OrderAcceptedCountTotal90',
       'ExemplarReturnedCountTotal7', 'ExemplarReturnedCountTotal30',
       'ExemplarReturnedCountTotal90', 'ExemplarReturnedValueTotal7',
       'ExemplarReturnedValueTotal30', '

In [8]:
from pymorphy3 import MorphAnalyzer
import re
from concurrent.futures import ThreadPoolExecutor

morph = MorphAnalyzer(lang='ru')

def lemmatize_batch(texts):
    results = []
    for text in texts:
        if not text:
            results.append('')
            continue
        
        tokens = re.findall(r'\b[а-яё]+\b', text.lower())
        lemmas = []
        for token in tokens:
            if len(token) > 1:
                parsed = morph.parse(token)[0]
                lemmas.append(parsed.normal_form)
        results.append(' '.join(lemmas))
    
    return results

def clean_column_optimized(series, batch_size=1000):
    series = series.fillna('')
    
    cleaned = series.apply(lambda x: BeautifulSoup(str(x), 'html.parser').get_text())\
                   .apply(lambda x: unicodedata.normalize('NFKC', x))\
                   .str.lower()\
                   .str.replace(r'[^\w\s]', ' ', regex=True)\
                   .str.replace(r'\s+', ' ', regex=True)\
                   .str.strip()
    
    texts = cleaned.tolist()
    lemmatized_texts = []
    
    with ThreadPoolExecutor() as executor:
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            lemmatized_batch = lemmatize_batch(batch)
            lemmatized_texts.extend(lemmatized_batch)
    
    return pd.Series(lemmatized_texts, index=series.index)

df['description'] = clean_column_optimized(df['description'])

C:\Users\Kostya\AppData\Local\Temp\ipykernel_7476\2618008016.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleaned = series.apply(lambda x: BeautifulSoup(str(x), 'html.parser').get_text())\


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import nltk
from nltk.corpus import stopwords

try:
    russian_stopwords = stopwords.words('russian')
except:
    nltk.download('stopwords')
    russian_stopwords = stopwords.words('russian')

def simple_tfidf_processing(df, text_column='description'):
    
    texts = df[text_column].fillna('').tolist()
    
    char_tfidf = TfidfVectorizer(
        analyzer='char',
        ngram_range=(3, 5),
        max_features=10000,
        min_df=2
    )
    
    X_char = char_tfidf.fit_transform(texts)
    char_features = [f'char_{f}' for f in char_tfidf.get_feature_names_out()]
    
    word_tfidf = TfidfVectorizer(
        analyzer='word', 
        ngram_range=(1, 3),
        max_features=10000,
        min_df=2,
        stop_words=russian_stopwords,
        lowercase=True,
        token_pattern=r'\b[а-яёa-z]+\b'
    )
    
    X_word = word_tfidf.fit_transform(texts)
    word_features = [f'word_{f}' for f in word_tfidf.get_feature_names_out()]
    
    char_df = pd.DataFrame(X_char.toarray(), columns=char_features, index=df.index)
    word_df = pd.DataFrame(X_word.toarray(), columns=word_features, index=df.index)
    
    result_df = pd.concat([df, char_df, word_df], axis=1)
    
    print(f"{len(char_features)} символьных фич")
    print(f"{len(word_features)} словных фич")
    
    return result_df, char_features + word_features

df, feature_names = simple_tfidf_processing(df)
print('Description is ready')
df, _ = simple_tfidf_processing(df, text_column='name_rus')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kostya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


MemoryError: Unable to allocate 14.7 GiB for an array with shape (197198, 10000) and data type float64

In [ ]:
list(df_processed.columns)

['id',
 'resolution',
 'brand_name',
 'description',
 'name_rus',
 'CommercialTypeName4',
 'rating_1_count',
 'rating_2_count',
 'rating_3_count',
 'rating_4_count',
 'rating_5_count',
 'comments_published_count',
 'photos_published_count',
 'videos_published_count',
 'PriceDiscounted',
 'item_time_alive',
 'item_count_fake_returns7',
 'item_count_fake_returns30',
 'item_count_fake_returns90',
 'item_count_sales7',
 'item_count_sales30',
 'item_count_sales90',
 'item_count_returns7',
 'item_count_returns30',
 'item_count_returns90',
 'GmvTotal7',
 'GmvTotal30',
 'GmvTotal90',
 'ExemplarAcceptedCountTotal7',
 'ExemplarAcceptedCountTotal30',
 'ExemplarAcceptedCountTotal90',
 'OrderAcceptedCountTotal7',
 'OrderAcceptedCountTotal30',
 'OrderAcceptedCountTotal90',
 'ExemplarReturnedCountTotal7',
 'ExemplarReturnedCountTotal30',
 'ExemplarReturnedCountTotal90',
 'ExemplarReturnedValueTotal7',
 'ExemplarReturnedValueTotal30',
 'ExemplarReturnedValueTotal90',
 'ItemVarietyCount',
 'ItemAvailab

In [ ]:
# Target Encoding
kf = KFold(n_splits=5, shuffle=True, random_state=42)
df['CommercialTypeName4_target'] = np.nan

for train_idx, val_idx in kf.split(df):
    means = df.iloc[train_idx].groupby('CommercialTypeName4')['resolution'].mean()
    df.iloc[val_idx, df.columns.get_loc('CommercialTypeName4_target')] = \
        df.iloc[val_idx]['CommercialTypeName4'].map(means).fillna(df['resolution'].mean())

In [ ]:
# Hashing Encoding
df['brand_name'] = df['brand_name'].fillna('MISSING')

hasher = HashingEncoder(cols=['brand_name'], n_components=12)
df_hashed = hasher.fit_transform(df[['brand_name']])

df = pd.concat([df, df_hashed], axis=1)

In [ ]:
df = df.drop(columns=['CommercialTypeName4', 'brand_name', 'description', 'name_rus'])

In [ ]:
df

,id,resolution,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,photos_published_count,videos_published_count,...,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11
0,159385,0,6.0,4.0,4.0,3.0,32.0,3.0,6.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,288616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
2,108090,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,415607,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
4,332391,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,329294,0,10.0,8.0,9.0,2.0,6.0,13.0,4.0,0.0,...,0,0,0,0,0,0,0,0,1,0
996,441110,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
997,382241,0,1.0,1.0,0.0,3.0,41.0,47.0,11.0,0.0,...,0,0,0,0,0,0,0,1,0,0
998,155636,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df.to_csv('../data/ml_ozon_counterfeit_train_preprocesed.csv', index=False)
